In [1]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00:00:0100:01


In [2]:
import ultralytics
import torch
from ultralytics import YOLO

print("Torch CUDA available:", torch.cuda.is_available())
print("Torch version:", torch.__version__)
print("Ultralytics version:", ultralytics.__version__)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch CUDA available: True
Torch version: 2.6.0+cu124
Ultralytics version: 8.3.194


In [3]:
# Mount your dataset (replace with your Kaggle dataset path)
!mkdir -p /kaggle/working
!cp -r /kaggle/input/pepper-dataset* /kaggle/working/

In [4]:
import os
from ultralytics import YOLO

# ------------------ Paths ------------------ #
BASE_DIR = "/kaggle/working/pepper-dataset"

dataset_yaml = os.path.join(BASE_DIR, "dataset", "data.yaml")
teacher_weights_dir = os.path.join(BASE_DIR, "models", "teacher")
os.makedirs(teacher_weights_dir, exist_ok=True)

# ------------------ Training Config ------------------ #
pretrained_model = "/kaggle/working/yolov8n-pose.pt"

epochs = 400   # adjust
batch_size = 16  # Kaggle GPU can handle higher than CPU
img_size = 640
best_model_path = os.path.join(teacher_weights_dir, "best.pt")

# ------------------ Train ------------------ #
def train_teacher():
    model = YOLO(pretrained_model)

    model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=os.path.join(BASE_DIR, "runs", "train_teacher"),
        name="teacher_pose",
        exist_ok=True,
        save=True,
        save_period=1,
        workers=2,  # Kaggle safe setting
        device=0    # force GPU
    )

    last_run_best = os.path.join(
        BASE_DIR, "runs", "best.pt")
    if os.path.exists(last_run_best):
        os.replace(last_run_best, best_model_path)
        print(f"✅ Best teacher model saved at {best_model_path}")
    else:
        print("⚠️ Best model not found, check training logs.")

if __name__ == "__main__":
    train_teacher()


Ultralytics 8.3.194 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/pepper-dataset/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=400, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=teacher_pose, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         93        154      0.652      0.604       0.63      0.537      0.648      0.606      0.629      0.621
                  Ripe         24         34      0.988      0.971       0.98      0.853      0.981      0.971       0.98      0.971
                Unripe         73        119      0.968       0.84      0.909      0.757      0.962      0.846      0.907      0.893
                  Stem          1          1          0          0          0          0          0          0          0          0
Speed: 0.4ms preprocess, 3.1ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to /kaggle/working/pepper-dataset/runs/train_teacher/teacher_pose
⚠️ Best model not found, check training logs.
